# Decorators : closure, variable scope, nonlocal...

## Everything you need to know

---------------------

source : "Fluent Python" by Luciano Ramalho

---------------------

### Decorators 101

Definition : a decorator is a callable that takes another function as argument (the decorated function). The decorator may perform some processing with the decorated function, and return it or replace it with another function or callable object.

Syntax and syntaxer

In [2]:
def decorate(func):
    print('It\'s decorate!')
    return func

In [7]:
@decorate
def target1():
    print('running target1()')
    
target1()

It's decorate!
running target1()


In [11]:
def target2():
    print('running target2()')
    
decorate(target2) # Not decorate(target2())
target2()

It's decorate!
running target2()


-------------

Here is a decorator that replaces a function with a different one.

In [14]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print('running target()')

In [15]:
target()

running inner()


In [16]:
target

<function __main__.deco.<locals>.inner()>

--------------------------------

### When Python executes decorators

![question pour un kahoot](assets/Kahoot_is_not_my_friend_anymore.png)

In [17]:
registry = []

def register(func):
    print(f'running register({func})')
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
@register
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')
    
def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()
    
if __name__ == '__main__':
    main()

running register(<function f1 at 0x7f11f17e4e50>)
running register(<function f2 at 0x7f11f0f2b0d0>)
running main()
registry -> [<function f1 at 0x7f11f17e4e50>, <function f2 at 0x7f11f0f2b0d0>]
running f1()
running f2()
running f3()


=> They run right after the decorated function is defined. That is usually at import time.

---------------------

In [ ]:
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        ifitem.quantity >= 20:
            discount += item.total() * .1
    return discount

@promotion
def large_orger(order):
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    return order.total() * .07 if len(distinct_items) >= 10 else 0

def best_promo(order):
    """Select best discount available"""
    return max(promo(order) for promo in promos)

--------------

Until now, our decorators don"t change the decorated function. To do so, we usually need to define an inner function and return it to replace the decorated function. Code that uses inner function almost always  depends on closures to operate correctly. And to understand closures, we need to look at how variables scopes work in Python.

### Variable Scope

In [18]:
def f1(a):
    print(a)
    print(b)
    
f1(3)

3


NameError: name 'b' is not defined

In [19]:
b = 6
f1(3)

3
6


In [20]:
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9
    
f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

The generate bytecode is different

In [21]:
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9
    
f3(3)
b

3
6


9

### Closures